Aufgaben zu async/await
=======================

Analysieren Sie die nachfolgend angegebenen Methoden. Lösen Sie auf Basis dieser
Methoden untenstehende Aufgaben.

In [1]:
async static Task<IEnumerable<T>> FilterAsync<T>(this IEnumerable<T> items, Func<T, bool> predicate) 
{
  var result = new List<T>();
  foreach (var item in items)
  {
    await Task.Delay(100);
    if (predicate(item))
    {
      result.Add(item);
    }
  }
  return result;
}

static bool IsPrime(int n) => n>=2 && !Enumerable.Range(2, (int)Math.Sqrt(n+1)-1).Any(i => n%i==0);

## Aufgabe 1
Ermitteln Sie alle Primzahlen im Bereich von 1 bis 20.

In [4]:
// TODO
async static Task<IEnumerable<int>> GetPrimesAsync()
{
    IEnumerable<int> primes = await Enumerable.Range(1, 20).FilterAsync(IsPrime);
    return primes;
}

IEnumerable<int> primes = await GetPrimesAsync();
Console.WriteLine(string.Join(", ", primes));

2, 3, 5, 7, 11, 13, 17, 19


## Aufgabe 2
Ermitteln Sie alle Primzahlen im Interval [1,20], indem Sie die
Primzahlenbestimmung *parallel* auf den Intervallen [1,10] und [11,20]
durchführen und anschließend die Teilergebnisse zusammenfügen
(`IEnumerable.Concat`). Vergleich Sie die Laufzeit dieser Variante mit jeder aus
Aufgabe 1.

In [5]:
// TODO
using System.Diagnostics;

async static Task<IEnumerable<int>> FilterPrimesParallelAsync()
{
    // Divide the range into two subranges
    var range1 = Enumerable.Range(1, 10);
    var range2 = Enumerable.Range(11, 10);

    // Process the ranges in parallel
    var task1 = range1.FilterAsync(IsPrime);
    var task2 = range2.FilterAsync(IsPrime);

    // Wait for both tasks to complete and concatenate the results
    await Task.WhenAll(task1, task2);
    return task1.Result.Concat(task2.Result);
}

static void CompareExecutionTimes()
{
    // Measure time for sequential filtering
    var stopwatch = Stopwatch.StartNew();
    var sequentialPrimes = GetPrimesAsync().Result;
    stopwatch.Stop();
    Console.WriteLine($"Sequential Primes: {string.Join(", ", sequentialPrimes)}");
    Console.WriteLine($"Sequential Time: {stopwatch.ElapsedMilliseconds} ms");

    // Measure time for parallel filtering
    stopwatch.Restart();
    var parallelPrimes = FilterPrimesParallelAsync().Result;
    stopwatch.Stop();
    Console.WriteLine($"Parallel Primes: {string.Join(", ", parallelPrimes)}");
    Console.WriteLine($"Parallel Time: {stopwatch.ElapsedMilliseconds} ms");
}

// Run the comparison
CompareExecutionTimes();

Sequential Primes: 2, 3, 5, 7, 11, 13, 17, 19
Sequential Time: 2118 ms
Parallel Primes: 2, 3, 5, 7, 11, 13, 17, 19
Parallel Time: 1051 ms
